In [1]:
import pandas as pd 
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_parquet(r'C:\Users\mauri\OneDrive\Escritorio\MLops\data\processed\modelo_dataset.parquet')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22257 entries, 0 to 22256
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   title      22257 non-null  object
 1   predictor  22257 non-null  object
dtypes: object(2)
memory usage: 347.9+ KB


Dejaremos comentado los codigos por tema de optimizacion para render, descomentar en local para probar.

In [52]:
# Crear el vectorizador TF-IDF
vectorizer = TfidfVectorizer(min_df=4, max_df=0.85, ngram_range=(1, 2), max_features=40000, dtype=np.float32)

# Transformar la columna 'overview' en una matriz TF-IDF
matriz = vectorizer.fit_transform(df['predictor'])

# Reductir la dimensionalidad con SVD.
svd = TruncatedSVD(n_components=400, random_state=42)
matriz_reducida = svd.fit_transform(matriz)
#matriz_reducida = matriz.tocsr()

#num_features = len(vectorizer.vocabulary_)

import psutil
import os

def get_memory_usage():
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss / (1024 * 1024)  # Convertir de bytes a MB

print(f"Uso de memoria: {get_memory_usage()} MB")
#print(f"El vectorizador está utilizando {num_features} características.")
#print(f"Dimensiones reducidas a {matriz_reducida.shape[1]} componentes.")

Uso de memoria: 411.11328125 MB


In [56]:
def recommendacion(title):
    # Encuentra el índice de la película
    idx = df.index[df['title'] == title].tolist()
    if not idx:
        return "Película no encontrada"
    idx = idx[0]
    
    # Calcular la matriz de similitud del coseno
    sim_scores = cosine_similarity(matriz_reducida[idx].reshape(1, -1), matriz_reducida)
    
    # Obtén los puntajes de similitud para la película seleccionada
    sim_scores = list(enumerate(sim_scores[idx]))
    
    # Ordena las películas basadas en la similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Obtén los índices de las películas más similares
    movie_indices = [i[0] for i in sim_scores[1:6]]  # 10 películas más similares
    
    return df['title'].iloc[movie_indices]

# Prueba la función de recomendación
print(recommendacion('toy story'))


import psutil
import os

def get_memory_usage():
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss / (1024 * 1024)  # Convertir de bytes a MB

print(f"Uso de memoria: {get_memory_usage()} MB")


TypeError: 'numpy.float32' object is not iterable